# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science <br><br> Module E - Project Group 34
###  By Paul-Emile Landrin, William Palmer, Victor Sheng, Royce Yap

<hr style="height:2pt">

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

# Dataset 1: NYC Capital Projects Data

## Part I: Data Pre-Processing

### 1. Import necessary libaries and read the dataset into a Pandas Dataframe

In [2]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import pandas as pd
import time
import datetime
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns

In [4]:
df_proj = pd.read_csv('../data/Capital_Projects.csv')
df_proj.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259 entries, 0 to 2258
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date Reported As Of      2259 non-null   object 
 1   PID                      2259 non-null   int64  
 2   Project Name             2259 non-null   object 
 3   Description              2150 non-null   object 
 4   Category                 2259 non-null   object 
 5   Borough                  1634 non-null   object 
 6   Managing Agency          2259 non-null   object 
 7   Client Agency            1644 non-null   object 
 8   Current Phase            2154 non-null   object 
 9   Design Start             2095 non-null   object 
 10  Budget Forecast          2153 non-null   float64
 11  Latest Budget Changes    1881 non-null   float64
 12  Total Budget Changes     2194 non-null   float64
 13  Forecast Completion      2137 non-null   object 
 14  Latest Schedule Changes 

### 2. Check and repair for missing values

Next, we explore the dataset to investigate the presence of missing values and if any of them can be repaired through imputation or otherwise. There seems to be an issue with there being more missing values for Budget Forecast vs. Total Budget Changes. We would expect there to be values for the total Budget Forecast, as long as a budget change was made.

In [4]:
df_proj.isnull().sum().sort_values(ascending=False)

Borough                    625
Client Agency              615
Latest Schedule Changes    393
Latest Budget Changes      378
Design Start               164
Forecast Completion        122
Description                109
Budget Forecast            106
Current Phase              105
Total Schedule Changes      73
Total Budget Changes        65
Managing Agency              0
Category                     0
Project Name                 0
PID                          0
Date Reported As Of          0
dtype: int64

Looking through the dataset, it appears that some of the projects have either their latest or earliest reported status with missing values in certain columns, possibly as a result of data collection. To fix this, we use the forward fill and backward fill functions in groupby to fill in the missing values based on the adjacent entries with the same project IDs.

In [5]:
# Fill in missing values for Project's Budget Forecast based on PID
df_proj['Budget Forecast']=df_proj.groupby(['PID'])['Budget Forecast'].ffill(1)
df_proj['Budget Forecast']=df_proj.groupby(['PID'])['Budget Forecast'].bfill(1)

# Fill in missing values for Project's Design Start based on PID
df_proj['Design Start']=df_proj.groupby(['PID'])['Design Start'].ffill(1)
df_proj['Design Start']=df_proj.groupby(['PID'])['Design Start'].bfill(1)

# Fill in missing values for Project's Schedule Completion based on PID
df_proj['Forecast Completion']=df_proj.groupby(['PID'])['Forecast Completion'].ffill(1)
df_proj['Forecast Completion']=df_proj.groupby(['PID'])['Forecast Completion'].bfill(1)

# Fill in missing values for Project's Total Budget Changes based on PID
df_proj['Total Budget Changes']=df_proj.groupby(['PID'])['Total Budget Changes'].ffill(1)
df_proj['Total Budget Changes']=df_proj.groupby(['PID'])['Total Budget Changes'].bfill(1)

# Fill in missing values for Project's Total Schedule Changes based on PID
df_proj['Total Schedule Changes']=df_proj.groupby(['PID'])['Total Schedule Changes'].ffill(1)
df_proj['Total Schedule Changes']=df_proj.groupby(['PID'])['Total Schedule Changes'].bfill(1)

In [6]:
# Additional operation for missing data
df_proj.loc[1462, df_proj.columns != 'Date Reported As Of'].replace(df_proj.loc[1461], inplace=True)
df_proj.loc[1462, "Latest Budget Changes"]=0

### 3. Change date-time variables to datetime type in Pandas

To allow for further data exploration and visualization, we change the date-time variables to datetime type and also sort the values by datetime and PID, such that the latest entry for each project appears first across all the same PID entries.

In [7]:
df_proj["Date Reported As Of"] = pd.to_datetime(df_proj["Date Reported As Of"])
df_proj["Design Start"] = pd.to_datetime(df_proj["Design Start"])
df_proj["Forecast Completion"] = pd.to_datetime(df_proj["Forecast Completion"])

In [8]:
df_proj.sort_values(by=["PID","Date Reported As Of"],ascending=[True,False], inplace=True)

In [9]:
df_proj.head(3)

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes
12,2019-09-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.805777e+08,-4664980.0,-4318643.37,2020-10-09,0.0,270.0
11,2019-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.852427e+08,142978.0,-4318643.37,2020-10-09,42.0,270.0
10,2019-01-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.850997e+08,8025707.0,-4318643.37,2020-08-28,31.0,270.0


In [10]:
# Code to change anomalous errors in entering data
df_proj.loc[df_proj["Forecast Completion"].dt.year==2108, "Forecast Completion"]=datetime.datetime(2018,10,21)

# Project reported 0 budget forecast in first entry before adjusting it; this is anomalous behavior
df_proj.loc[df_proj["PID"]==863, "Budget Forecast"]= 160090000
df_proj.loc[df_proj["PID"]==863, "Latest Budget Changes"]= 0
df_proj.loc[df_proj["PID"]==863, "Total Budget Changes"]= 0

### 4. Create Columns for Original Budget, Completion Date and Expected Duration

In [11]:
# Create list of unique projects 
df_proj_unique = df_proj.copy()
df_proj_unique.drop_duplicates(subset ="PID", keep = "first", inplace = True) 

In [12]:
# Calculate Original values by deducting total changes for most recent forecast
df_proj_unique["Original Budget"]=df_proj_unique["Budget Forecast"] - df_proj_unique["Total Budget Changes"]
df_proj_unique["Original Completion"]=df_proj_unique["Forecast Completion"] - pd.to_timedelta(df_proj_unique["Total Schedule Changes"], unit="D")
df_proj_unique["Original Expected Duration"]=(df_proj_unique["Original Completion"] - df_proj_unique["Design Start"])/pd.Timedelta(days=1)

In [13]:
#Left join between original dataset and unique project dataset
df_proj_new = pd.merge(df_proj,
                 df_proj_unique[['PID','Original Budget', 'Original Completion', 'Original Expected Duration']],
                 on='PID', 
                 how='left')

In [14]:
print(f"Original dataset size: {df_proj.shape}\nNew dataset size: {df_proj_new.shape}\nUnique project size: {df_proj_unique.shape}")

Original dataset size: (2259, 16)
New dataset size: (2259, 19)
Unique project size: (378, 19)


### 5. Fill in blank values for Latest Budget Changes and Latest Schedule Changes

We observe that for some projects, their very first entry has NaN for latest budget and latest schedule changes. We fix that by assigning a 0 to those entries instead.

In [15]:
df_proj_new.loc[df_proj_new["Latest Budget Changes"].isnull() & df_proj_new["Total Budget Changes"].notnull(),["Latest Budget Changes"]]=0

In [16]:
df_proj_new.loc[df_proj_new["Latest Schedule Changes"].isnull() & df_proj_new["Total Schedule Changes"].notnull(),["Latest Schedule Changes"]]=0

### 6. Create Columns for Scaled Budget and Schedule Changes

In [17]:
# Create columns for scaled budget and schedule changes, relative to original duration
df_proj_new["Latest Budget Changes (Scaled to Original)"] = df_proj_new["Latest Budget Changes"] / df_proj_new["Original Budget"]
df_proj_new["Total Budget Changes (Scaled to Original)"] = df_proj_new["Total Budget Changes"] / df_proj_new["Original Budget"]
df_proj_new["Latest Schedule Changes (Scaled to Original)"] = df_proj_new["Latest Schedule Changes"] / df_proj_new["Original Expected Duration"]
df_proj_new["Total Schedule Changes (Scaled to Original)"] = df_proj_new["Total Schedule Changes"] / df_proj_new["Original Expected Duration"]

In [18]:
df_proj_new.head(3)

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Original Budget,Original Completion,Original Expected Duration,Latest Budget Changes (Scaled to Original),Total Budget Changes (Scaled to Original),Latest Schedule Changes (Scaled to Original),Total Schedule Changes (Scaled to Original)
0,2019-09-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.805777e+08,-4664980.0,-4318643.37,2020-10-09,0.0,270.0,184896360.0,2020-01-13,2303.0,-0.025230,-0.023357,0.000000,0.117238
1,2019-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.852427e+08,142978.0,-4318643.37,2020-10-09,42.0,270.0,184896360.0,2020-01-13,2303.0,0.000773,-0.023357,0.018237,0.117238
2,2019-01-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.850997e+08,8025707.0,-4318643.37,2020-08-28,31.0,270.0,184896360.0,2020-01-13,2303.0,0.043407,-0.023357,0.013461,0.117238


### 7. Create Columns for Total Budget and Schedule Change Frequencies (i.e. number of non-zero updates)

In [19]:
budget_change = df_proj_new[df_proj_new["Latest Budget Changes"]!=0].groupby(by="PID")["Latest Budget Changes"].count()
budget_change = budget_change.rename("Total Frequency of Budget Changes")
df_proj_new = pd.merge(df_proj_new,
                 budget_change,
                 on='PID',
                 how='left')

In [20]:
schedule_change = df_proj_new[df_proj_new["Latest Schedule Changes"]!=0].groupby(by="PID")["Latest Schedule Changes"].count()
schedule_change = schedule_change.rename("Total Frequency of Schedule Changes")
df_proj_new = pd.merge(df_proj_new,
                 schedule_change,
                 on='PID',
                 how='left')

In [21]:
df_proj_new["Total Frequency of Budget Changes"].fillna(0, inplace=True)
df_proj_new["Total Frequency of Schedule Changes"].fillna(0, inplace=True)

In [22]:
df_proj_new.shape

(2259, 25)

### 8. Create Columns for Cumulative Budget and Schedule Changes as of Date Reported

In [23]:
df_proj_new.sort_values(by=["PID","Date Reported As Of"],ascending=[True,True], inplace=True)

In [24]:
df_proj_new["Cumulative Budget Changes ($)"]=df_proj_new.groupby(by="PID")["Latest Budget Changes"].cumsum()
df_proj_new["Cumulative Schedule Changes (days)"]=df_proj_new.groupby(by="PID")["Latest Schedule Changes"].cumsum()

In [25]:
df_proj_new.loc[df_proj_new["Latest Budget Changes"] !=0, 'Budget_Change_Boolean'] = 1
df_proj_new.loc[df_proj_new["Budget_Change_Boolean"] !=1, 'Budget_Change_Boolean'] = 0
df_proj_new.loc[df_proj_new["Latest Schedule Changes"] !=0, 'Schedule_Change_Boolean'] = 1
df_proj_new.loc[df_proj_new["Schedule_Change_Boolean"] !=1, 'Schedule_Change_Boolean'] = 0

In [26]:
df_proj_new["Cumulative Budget Changes (no. of times)"]=df_proj_new.groupby(by="PID")["Budget_Change_Boolean"].cumsum()
df_proj_new["Cumulative Schedule Changes (no. of times)"]=df_proj_new.groupby(by="PID")["Schedule_Change_Boolean"].cumsum()

In [27]:
print(f"Original dataset size: {df_proj.shape}\nNew dataset size: {df_proj_new.shape}")

Original dataset size: (2259, 16)
New dataset size: (2259, 31)


We see that now, by extracting additional meta-information on the projects, we are able to increase the number of variables from 16 to 31.

### 7. Create Dataset excluding Projects with Null Values for Budget and Schedule Changes

There are projects for which there is insufficient data on budget and schedule. For those projects with missing total budget and schedule change information, we drop them, since they cannot be used for predictive purposes. With this we drop 65 entries.

In [28]:
#We create new columns to indicate if the budget and schedule changes are null. 
#We then find the sum of all values with the same PID.
#For PIDs with values of 0, indicating all budget changes are null, we drop them
df_proj_new.loc[df_proj_new["Latest Budget Changes"].notnull(), 'Null_Budget_Change_Values'] = 1
df_proj_new.loc[df_proj_new["Latest Budget Changes"].isnull(), 'Null_Budget_Change_Values'] = 0
df_proj_new.loc[df_proj_new["Latest Schedule Changes"].notnull(), 'Null_Schedule_Change_Values'] = 1
df_proj_new.loc[df_proj_new["Latest Schedule Changes"].isnull(), 'Null_Schedule_Change_Values'] = 0

In [29]:
df_proj_new['Sum_of_Budget_Nulls'] = df_proj_new.groupby('PID')['Null_Budget_Change_Values'].transform(np.sum)
df_proj_new['Sum_of_Schedule_Nulls'] = df_proj_new.groupby('PID')['Null_Schedule_Change_Values'].transform(np.sum)
df_proj_exclude_nulls=df_proj_new[(df_proj_new['Sum_of_Budget_Nulls']!=0) | (df_proj_new['Sum_of_Schedule_Nulls']!=0)]

In [30]:
df_proj_new.drop(['Null_Budget_Change_Values', 'Null_Schedule_Change_Values', 'Sum_of_Budget_Nulls', 'Sum_of_Schedule_Nulls'], axis=1, inplace=True)
df_proj_exclude_nulls = df_proj_exclude_nulls.drop(['Null_Budget_Change_Values', 'Null_Schedule_Change_Values', 'Sum_of_Budget_Nulls', 'Sum_of_Schedule_Nulls'], axis=1)

In [31]:
print(f"New dataset size (all rows): {df_proj_new.shape}\nNew dataset size (without nulls): {df_proj_exclude_nulls.shape}")

New dataset size (all rows): (2259, 31)
New dataset size (without nulls): (2194, 31)


In [32]:
df_proj_exclude_nulls.isnull().sum().sort_values(ascending=False)

Client Agency                                   602
Borough                                         565
Description                                      94
Current Phase                                    93
Latest Schedule Changes (Scaled to Original)     65
Original Expected Duration                       65
Total Schedule Changes (Scaled to Original)      65
Design Start                                     58
Original Completion                              24
Forecast Completion                               9
Cumulative Schedule Changes (days)                8
Latest Schedule Changes                           8
Total Schedule Changes                            8
Latest Budget Changes (Scaled to Original)        1
Category                                          0
Budget Forecast                                   0
Managing Agency                                   0
Project Name                                      0
PID                                               0
Cumulative S

In [33]:
df_proj_exclude_nulls.to_csv("df_project_listing.csv")

In [34]:
#Obtain first and last entry for each project
df_proj_exclude_nulls["Latest_Report"]=df_proj_exclude_nulls.groupby('PID')['Date Reported As Of'].transform(np.max)
df_proj_exclude_nulls["Earliest_Report"]=df_proj_exclude_nulls.groupby('PID')['Date Reported As Of'].transform(np.min)

## Part II: Creating Snapshot by Project ID

### 1. Create new dataset with one row for each Project ID ("PID")

To understand the projects better, we investigate projects at a Project-Level by keeping the most recent (sorted by "Date Reported As Of") row for each project ID. This produces a dataframe with 378 rows, each corresponding to a unique PID.

In [35]:
df_proj_unique_exclude_nulls = df_proj_exclude_nulls.copy()
df_proj_unique_exclude_nulls.drop_duplicates(subset ="PID", keep = "last", inplace = True) 

In [36]:
df_proj_unique_exclude_nulls.shape

(314, 33)

In [37]:
df_proj_unique_exclude_nulls.head(3)

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Original Budget,Original Completion,Original Expected Duration,Latest Budget Changes (Scaled to Original),Total Budget Changes (Scaled to Original),Latest Schedule Changes (Scaled to Original),Total Schedule Changes (Scaled to Original),Total Frequency of Budget Changes,Total Frequency of Schedule Changes,Cumulative Budget Changes ($),Cumulative Schedule Changes (days),Budget_Change_Boolean,Schedule_Change_Boolean,Cumulative Budget Changes (no. of times),Cumulative Schedule Changes (no. of times),Latest_Report,Earliest_Report
0,2019-09-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.805777e+08,-4664980.0,-4318643.37,2020-10-09,0.0,270.0,184896360.0,2020-01-13,2303.0,-0.025230,-0.023357,0.000000,0.117238,13.0,12.0,-4318643.0,270.0,1.0,0.0,13.0,12.0,2019-09-01,2014-05-01
13,2019-09-01,7,Bowery Bay Waste Water Treatment Plant Main Se...,The existing Main Sewage Pumps have deteriorat...,Wastewater Treatment,Queens,DEP,DEP,Construction,2013-05-01,6.879846e+07,1065884.0,15305457.00,2020-02-14,88.0,685.0,53493000.0,2018-03-31,1795.0,0.019926,0.286121,0.049025,0.381616,14.0,12.0,15305457.0,685.0,1.0,1.0,14.0,12.0,2019-09-01,2013-09-01
28,2019-09-01,18,Croton Falls Pumping Station Construction,This project will construct a new pumping stat...,Water Supply,Carmel,DEP,DEP,Construction,2003-08-22,6.787161e+07,-1399611.0,1255175.02,2020-03-10,91.0,1014.0,66616435.0,2017-05-31,5031.0,-0.021010,0.018842,0.018088,0.201550,13.0,8.0,1255174.0,1014.0,1.0,1.0,13.0,8.0,2019-09-01,2013-09-01


In [38]:
df_proj_unique_exclude_nulls.to_csv("df_project_unique.csv")

## Part III: Creating Time Series Data

### 1. Create dataset with an entry for every month between the earliest and latest reporting date of a project

For each month, we include budget and schedule changes only in cases where there were reports of budget/schedule changes. Otherwise we leave it as 0. This sequential data can be used for building sequential models.

In [39]:
df_time_series = df_proj_unique_exclude_nulls.copy().reset_index()

In [40]:
#First we create a dictionary with each key representing a PID and value representing a Panda series containing range of dates
#The range of dates is from the earliest date an entry was made to the latest date, with a monthly interval
pid_dict={}

for idx in range(len(df_time_series)):
    pid_dict[df_time_series.loc[idx, ["PID"]].get("PID")]=pd.date_range(df_time_series.loc[idx, ["Earliest_Report"]].get("Earliest_Report"),df_time_series.loc[idx, ["Latest_Report"]].get("Latest_Report"),freq="MS") 

In [41]:
#We then create sequential data by appending each date range to a dataframe
df_sequence = pd.DataFrame()

for i in pid_dict.keys():
    df_temp = pd.DataFrame()
    df_temp["Date Reported As Of"]=pid_dict[i]
    df_temp["PID"]=i
    df_sequence = df_sequence.append(df_temp, ignore_index=True)
df_sequence

,Date Reported As Of,PID
0,2014-05-01,3
1,2014-06-01,3
2,2014-07-01,3
3,2014-08-01,3
4,2014-09-01,3
...,...,...
9548,2019-06-01,956
9549,2019-07-01,956
9550,2019-08-01,956
9551,2019-09-01,956


In [42]:
#This dataframe skeleton is then merged with the original data, including entries only in cases where there were reports
df_sequence_new = pd.merge(df_sequence,
                 df_proj_exclude_nulls[['PID',"Date Reported As Of", 'Latest Schedule Changes', "Latest Schedule Changes (Scaled to Original)", 'Latest Budget Changes', "Latest Budget Changes (Scaled to Original)"]],
                 on=['PID',"Date Reported As Of"], 
                 how='left')

#For months without entries (of which we should expect a lot), we fill them with 0 (assuming no changes with no reports)
df_sequence_new = df_sequence_new.fillna(0)

In [43]:
df_sequence_new.shape

(9553, 6)

In [44]:
df_sequence_new.head(10)

,Date Reported As Of,PID,Latest Schedule Changes,Latest Schedule Changes (Scaled to Original),Latest Budget Changes,Latest Budget Changes (Scaled to Original)
0,2014-05-01,3,1.0,0.000434,-1915400.0,-0.010359
1,2014-06-01,3,0.0,0.000000,0.0,0.000000
2,2014-07-01,3,0.0,0.000000,0.0,0.000000
3,2014-08-01,3,0.0,0.000000,0.0,0.000000
4,2014-09-01,3,0.0,0.000000,0.0,0.000000
5,2014-10-01,3,0.0,0.000000,0.0,0.000000
6,2014-11-01,3,0.0,0.000000,0.0,0.000000
7,2014-12-01,3,0.0,0.000000,0.0,0.000000
8,2015-01-01,3,0.0,0.000000,0.0,0.000000
9,2015-02-01,3,187.0,0.081198,-14229191.0,-0.076958


In [45]:
df_sequence_new.to_csv("df_sequence.csv")